In [1]:
import torch
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3090'

In [2]:
import torchvision

In [3]:
import cv2

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [5]:
from glob import glob

In [6]:
images = glob("/home/e/Downloads/Dataset/IDC_regular_ps50_idx5/**/*.png", recursive=True)

In [7]:
len(images)

277524

In [8]:
images[0]

'/home/e/Downloads/Dataset/IDC_regular_ps50_idx5/9383/1/9383_idx5_x1801_y651_class1.png'

In [9]:
import fnmatch
negative = fnmatch.filter(images, "*class0.png")
positive = fnmatch.filter(images, "*class1.png")


In [10]:
from tqdm import tqdm

In [11]:
y = []
for img in tqdm(images):
    if img[-5]=="0":
        y.append(0)
    elif img[-5] == "1":
        y.append(1)
    else:
        print(img, img[-5])

100%|██████████████████████████████| 277524/277524 [00:00<00:00, 3094078.09it/s]


In [12]:
len(y)

277524

In [13]:
import pandas as pd

In [14]:
images_dataset = pd.DataFrame()
images_dataset["images"] = images
images_dataset["labels"] = y
images_dataset.head()

,images,labels
0,/home/e/Downloads/Dataset/IDC_regular_ps50_idx...,1
1,/home/e/Downloads/Dataset/IDC_regular_ps50_idx...,1
2,/home/e/Downloads/Dataset/IDC_regular_ps50_idx...,1
3,/home/e/Downloads/Dataset/IDC_regular_ps50_idx...,1
4,/home/e/Downloads/Dataset/IDC_regular_ps50_idx...,1


In [15]:
images_dataset.groupby("labels")["labels"].count()

labels
0    198738
1     78786
Name: labels, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
train, val = train_test_split(images_dataset, stratify=images_dataset.labels, test_size=0.2)
print(len(train), len(val))

222019 55505


In [18]:
class MyDataset(Dataset):
    def __init__(self, df_data,transform=None):
        super().__init__()
        self.df = df_data.values
        
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path,label = self.df[index]
        
        image = cv2.imread(img_path)
        image = cv2.resize(image, (50,50))
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [19]:
epochs = 10
classes = 2
batch_size = 128
alpha = 0.002

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [20]:
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

dataset_train = MyDataset(df_data=train, transform=trans_train)
dataset_valid = MyDataset(df_data=val,transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)

In [21]:
class CNN(nn.Module):
    def __init__(self):
        # ancestor constructor call
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avg = nn.AvgPool2d(7)
        self.fc = nn.Linear(512 * 1 * 1, 2) # !!!
        
    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x)))) # first convolutional layer then batchnorm, then activation then pooling layer.
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
        x = self.pool(F.leaky_relu(self.bn5(self.conv5(x))))
        x = self.avg(x)
        x = x.view(-1, 512 * 1 * 1) # !!!
        x = self.fc(x)
        return x

In [22]:
model = CNN().to(device)

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=alpha)

In [24]:
total_step = len(loader_train)
for epoch in range(epochs):
    for i, (images, labels) in tqdm(enumerate(loader_train)):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, epochs, i+1, total_step, loss.item()))

100it [00:20,  4.97it/s]

Epoch [1/10], Step [100/1735], Loss: 0.3642


200it [00:39,  5.02it/s]

Epoch [1/10], Step [200/1735], Loss: 0.3346


301it [00:59,  4.78it/s]

...

1601it [04:38,  5.48it/s]

Epoch [10/10], Step [1600/1735], Loss: 0.2851


1701it [04:55,  5.58it/s]

Epoch [10/10], Step [1700/1735], Loss: 0.2264


1735it [05:00,  5.77it/s]


In [25]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
confusion_matrix = torch.zeros(2, 2)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
                 
    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the test images: 90.53959102783533 %
